In [1]:
import torch
from torch import nn
import torchvision
import torchvision.datasets as datasets
from torch.autograd import Variable
from torchvision import transforms
from PIL import Image
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

mnist_trainset = datasets.MNIST(root='.', train=True, download=True, transform=transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ))
loader = torch.utils.data.DataLoader(mnist_trainset,batch_size=64,num_workers=16)
num_epochs = 300
print(device)

cuda


In [0]:
class Generator(nn.Module):
  def __init__(self,latent_dim,image_dim):
    super(Generator,self).__init__()
    self.latent_dim = latent_dim
    self.image_dim = image_dim
    def block(in_feat, out_feat, normalize=True):
        layers = [nn.Linear(in_feat, out_feat)]
        if normalize:
            layers.append(nn.BatchNorm1d(out_feat, 0.8))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        return layers

    self.model = nn.Sequential(
        *block(latent_dim, 128, normalize=False),
        *block(128, 256),
        *block(256, 512),
        *block(512, 1024),
        nn.Linear(1024, image_dim**2),
        nn.Tanh()
    )
  def forward(self,x):
    x = x.view(-1,self.latent_dim)
    out = self.model(x)
    out = out.view(-1,self.image_dim, self.image_dim)
    return out

In [0]:
class Discriminator(nn.Module):
    def __init__(self,image_shape):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(image_shape ** 2, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [0]:
gen = Generator(128,28).cuda()
discrim = Discriminator(28).cuda()
adversarial_loss = nn.BCELoss()

In [6]:

gen_optim = torch.optim.Adam(gen.parameters(),lr=1e-4)
discrim_optim = torch.optim.Adam(discrim.parameters(),lr=1e-4)
latent_dim = 128
for epoch in range(num_epochs):
  print(epoch)
  for i, data in enumerate(loader):
    img, _ = data
    img = Variable(img.detach().to(device))
    valid = Variable(torch.Tensor(img.shape[0],1).fill_(1.0),requires_grad=False).cuda()
    invalid = Variable(torch.Tensor(img.shape[0],1).fill_(0.0),requires_grad=False).cuda()
    
    # Train Discriminator
    discrim_optim.zero_grad()
    fake_imgs = gen(torch.randn((img.shape[0],latent_dim)).to(device)).detach()
    real_out = discrim(img)
    fake_out = discrim(fake_imgs)
    real_loss = adversarial_loss(real_out,valid)
    fake_loss = adversarial_loss(fake_out,invalid)
    d_loss = (real_loss + fake_loss) / 2
    d_loss.backward()
    discrim_optim.step()

    # Train Generator
    gen_optim.zero_grad()
    noise = Variable(torch.Tensor(np.random.normal(0, 1, (img.shape[0], latent_dim)))).to(device)
    out = gen(noise)
    discrim_out = discrim(out)
    g_loss = adversarial_loss(discrim_out,valid)
    
    g_loss.backward()
    gen_optim.step()

    if epoch % 5 == 0 and i % 100 == 0:
      print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, num_epochs, i, len(loader), d_loss.item(), g_loss.item())
        )

0
[Epoch 0/300] [Batch 0/938] [D loss: 0.684732] [G loss: 0.678743]
[Epoch 0/300] [Batch 100/938] [D loss: 0.442549] [G loss: 0.861359]
[Epoch 0/300] [Batch 200/938] [D loss: 0.259801] [G loss: 1.581638]
[Epoch 0/300] [Batch 300/938] [D loss: 0.535804] [G loss: 1.070444]
[Epoch 0/300] [Batch 400/938] [D loss: 0.701955] [G loss: 0.454073]
[Epoch 0/300] [Batch 500/938] [D loss: 0.446569] [G loss: 1.633453]
[Epoch 0/300] [Batch 600/938] [D loss: 0.241232] [G loss: 2.955877]
[Epoch 0/300] [Batch 700/938] [D loss: 0.246619] [G loss: 2.038481]
[Epoch 0/300] [Batch 800/938] [D loss: 0.437409] [G loss: 1.303104]
[Epoch 0/300] [Batch 900/938] [D loss: 0.297826] [G loss: 1.993575]
1
2
3
4
5
[Epoch 5/300] [Batch 0/938] [D loss: 0.148640] [G loss: 4.210704]
[Epoch 5/300] [Batch 100/938] [D loss: 0.079359] [G loss: 5.269105]
[Epoch 5/300] [Batch 200/938] [D loss: 0.081690] [G loss: 5.812726]
[Epoch 5/300] [Batch 300/938] [D loss: 0.045823] [G loss: 4.859176]
[Epoch 5/300] [Batch 400/938] [D loss: 0

In [3]:
gen.eval()
images = []
for i in range(20):
  pil = transforms.ToPILImage(mode='L')(img.cpu().view(-1,28,28))
  img = gen(torch.randn((1,latent_dim)).to(device))
  images.append(images)


NameError: ignored

In [2]:
import os

os.mkdir("generated_images")
i = 1
for img in images:
  img.save("generated_images/img_{}.jpg".format(i))
  i += 1

FileExistsError: ignored